# Objetivo 

Avaliar o modelo enviado pelo Henrique e checar o quão realista ele é em relação a modelos curados por especialistas.

### Genoma alvo 
- Phanerodontia chrysosporium
- Arquivo: `carveme_chrys.sbml`

### Modelos curados
- Infomações sobre o organismo: https://www.ncbi.nlm.nih.gov/datasets/taxonomy/2822231/names/
- GEM: https://www.ebi.ac.uk/biomodels/MODEL1302010025

### Peguei os genomas próximos 
- https://www.ncbi.nlm.nih.gov/datasets/genome/?taxon=396331 
- Reconstruí a árvore para ver se tinha algum outro que dava para usar
  
  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cobra

# Avaliando genoma alvo 

In [2]:
modelo_curado = "/home/gustavotamasco/Projects/Colaborativos/ModelagemHenrique/Curado/MODEL1302010025_url.xml"
model = cobra.io.read_sbml_model(modelo_curado)
model

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R08505>'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R08505>'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R07889>'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R07889>'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R07888>'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R07888>'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R08698>'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R08698>'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R01917>'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R01917>'
Missing lower flux bound set to '-1000.0' fo

Name,MODEL1302010025
Memory address,71ad42c4c670
Number of metabolites,1964
Number of reactions,2107
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,cytosol


### 1. Verificações gerais


**Meio de cultura**

In [3]:
model.medium

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


{}

> Nota: Como o modelo não tem reações de transporte, não é possível inferir o meio de cultura. Temos que adicionar as básicas para isso.
>
> Como podemos observar abaixo, o modelo usa informações do KEGG para definir as reações e compostos


In [4]:
for index, reac in enumerate(model.reactions):
    if index < 10:
        print(reac.id, reac.reaction)

R08505 C00005 + C00007 + 2.0 C00080 + C00187 <=> C00001 + C00006 + C15610
R07889 2.0 C00080 + C16329 <=> C00001 + C16328
R07888 C00016 + C16327 <=> C01352 + C16328
R08698 C00001 + C00080 + C02218 <=> C00014 + C00022
R01917 C00002 + C00051 + C00315 <=> C00008 + C00009 + C05730
R06338 C00007 + C00026 + C11864 <=> C00011 + C00042 + C11868
R01913 2.0 C00001 + C00314 + C01353 <=> 4.0 C00080 + C06056 + C06257
R06336 C00007 + C00026 + C11863 <=> C00011 + C00042 + C11864
R06337 C00007 + C00026 + C11863 <=> C00011 + C00042 + C11865
R00426 C00001 + C00044 <=> C00013 + 2.0 C00080 + C00144


In [5]:
for index, met in enumerate(model.metabolites):
    if index < 10:
        print(met.id, met.name, met.formula)

C04509 di-trans,poly-cis-Decaprenyl diphosphate [cytosol] None
C04506 Protein C-terminal S-farnesyl-L-cysteine [cytosol] None
C01755 Thiocyanate [cytosol] None
C01024 Hydroxymethylbilane [cytosol] None
C01751 Resorcinol [cytosol] None
C04500 N-Acetyl-D-glucosaminyldiphosphodolichol [cytosol] None
C04501 N-Acetyl-alpha-D-glucosamine 1-phosphate [cytosol] None
C04874 2-Amino-4-hydroxy-6-(D-erythro-1,2,3-trihydroxypropyl)-7,8- [cytosol] None
C16680 all-trans-5,6-Epoxyretinoic acid [cytosol] None
C16681 9-cis-Retinal [cytosol] None


**Criando BOF segundo paper**

In [6]:
bof_df = pd.read_csv(
    "/home/gustavotamasco/Projects/Colaborativos/ModelagemHenrique/Curado/KEGG_Compounds.csv"
)
bof_df.head()

,KEGG Compound,Coefficient,Name
0,C00965,-1.1348,"1,3-beta-D-Glucan"
1,C00041,-0.4588,ala-L
2,C00020,-0.0460,amp
3,C00062,-0.1607,arg-L
4,C00152,-0.1017,asn-L


In [7]:
from cobra import Reaction, Metabolite

In [11]:
# Criando BOF

# Cria a reação
reaction_bof = Reaction("BOF")

# Agora pegamos os metabólitos que compõem a reação

all_metabolites = {}
for metabolite in bof_df["KEGG Compound"]:
    c_met = model.metabolites.get_by_id(metabolite)
    all_metabolites[c_met] = bof_df[bof_df["KEGG Compound"] == metabolite][
        "Coefficient"
    ].values[0]


reaction_bof.add_metabolites(all_metabolites)

# Adiciona a reação ao modelo
model.add_reactions([reaction_bof])

In [12]:
model.reactions.get_by_id("BOF")

Reaction identifier,BOF
Name,
Memory address,0x71acb4a42160
Stoichiometry,59.276 C00001 + 59.276 C00002 + 1e-06 C00003 + 1e-06 C00010 + 1e-06 C00016 + 0.046 C00020 + 0.3018 C00025 + 0.2904 C00037 + 0.4588 C00041 + 0.2862 C00047 + 0.2975 C00049 + 1e-06 C00051 + 0.0447... 59.276 H2O [cytosol] + 59.276 ATP [cytosol] + 1e-06 NAD+ [cytosol] + 1e-06 CoA [cytosol] + 1e-06 FAD [cytosol] + 0.046 AMP [cytosol] + 0.3018 L-Glutamate [cytosol] + 0.2904 Glycine [cytosol] +...
GPR,
Lower bound,0.0
Upper bound,1000.0


**Adicionando o meio agora**

In [13]:
media_path = "/home/gustavotamasco/Projects/Colaborativos/ModelagemHenrique/Curado/Media_Compounds.csv"
paper_media_df = pd.read_csv(media_path)

paper_media_df.head()

,Name,KEGG compound,Coefficient
0,Water,C00001,1000.0
1,Glucose,C00031,22.6
2,Bicarbonate,C00288,100.0
3,Diphosphate,C00013,100.0
4,Iron,C00023,100.0


> primeiro precisa ajustar os bounds

In [24]:
bounds = pd.read_csv("./Curado/bounds.csv", index_col=0)
bounds.head()

,LowerBound,UpperBound
ReactionId,,
R00001,-1000,1000
R00004,0,1000
R00005,0,0
R00006,-1000,0
R00008,-1000,1000


# TODO: ajustar os bounds das reacoes
# TODO: adicionar o meio de cultura

In [26]:
for reaction in model.reactions:
    tg_reaction = model.reactions.get_by_id(reaction.id)
    print(tg_reaction)
    bounds_val = bounds[bounds.index == reaction.id]
    print(bounds_val)
    # tg_reaction.bounds = (bounds[bounds["reaction"] == reaction.id][["LowerBound", "UpperBound"]].values[0])

R08505: C00005 + C00007 + 2.0 C00080 + C00187 <=> C00001 + C00006 + C15610
Empty DataFrame
Columns: [LowerBound, UpperBound]
Index: []
R07889: 2.0 C00080 + C16329 <=> C00001 + C16328
            LowerBound  UpperBound
ReactionId                        
R07889           -1000           0
R07888: C00016 + C16327 <=> C01352 + C16328
            LowerBound  UpperBound
ReactionId                        
R07888           -1000           0
R08698: C00001 + C00080 + C02218 <=> C00014 + C00022
            LowerBound  UpperBound
ReactionId                        
R08698               0        1000
R01917: C00002 + C00051 + C00315 <=> C00008 + C00009 + C05730
            LowerBound  UpperBound
ReactionId                        
R01917           -1000           0
R06338: C00007 + C00026 + C11864 <=> C00011 + C00042 + C11868
Empty DataFrame
Columns: [LowerBound, UpperBound]
Index: []
R01913: 2.0 C00001 + C00314 + C01353 <=> 4.0 C00080 + C06056 + C06257
            LowerBound  UpperBound
ReactionId 

In [17]:
model.metabolites.get_by_id("C04509").summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


# Avaliando modelo enviado

In [2]:
h_model = "/home/gustavotamasco/Projects/Colaborativos/ModelagemHenrique/GEM_test/carveme_chrys.sbml"
model = cobra.io.read_sbml_model(h_model)
model

Adding exchange reaction EX_14glucan_e with default bounds for boundary metabolite: 14glucan_e.
Adding exchange reaction EX_25dkglcn_e with default bounds for boundary metabolite: 25dkglcn_e.
Adding exchange reaction EX_2m35mdntha_e with default bounds for boundary metabolite: 2m35mdntha_e.
Adding exchange reaction EX_2mbald_e with default bounds for boundary metabolite: 2mbald_e.
Adding exchange reaction EX_35dnta_e with default bounds for boundary metabolite: 35dnta_e.
Adding exchange reaction EX_3mb_e with default bounds for boundary metabolite: 3mb_e.
Adding exchange reaction EX_4abut_e with default bounds for boundary metabolite: 4abut_e.
Adding exchange reaction EX_4hphac_e with default bounds for boundary metabolite: 4hphac_e.
Adding exchange reaction EX_5mdru1p_e with default bounds for boundary metabolite: 5mdru1p_e.
Adding exchange reaction EX_LalaDgluMdap_e with default bounds for boundary metabolite: LalaDgluMdap_e.
Adding exchange reaction EX_Larab_e with default bounds fo

Name,augustus
Memory address,7e6180129bb0
Number of metabolites,746
Number of reactions,1068
Number of genes,523
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, extracellular space, periplasm"


**qual a BOF**

In [12]:
bof = model.reactions.get_by_id("Growth")
bof

Reaction identifier,Growth
Name,Biomass reaction
Memory address,0x7e60f2adc7c0
Stoichiometry,0.000223 10fthf_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 0.0001... 0.000223 10-Formyltetrahydrofolate + 0.513689 L-Alanine + 0.000223 S-Adenosyl-L-methionine + 0.295792 L-Arginine + 0.241055 L-Asparagine + 0.241055 L-Aspartate + 54.124831 ATP C10H12N5O13P3 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
bof.reaction

'0.000223 10fthf_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 0.0001 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 48.601527 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 0.0001 mql8_c + 0.001831 nad_c + 0.000447 nadp_c + 0.185265 phe__L_c + 0.221055 pro__L_c + 0.000223 pydx5p_c + 0.000223 ribflv_c + 0.215792 ser__L_c + 0.004338 so4_c + 0.000223 thf_c + 0.000223 thmpp_c + 0.253687 thr__L_c + 0.056843 trp__L_c + 0.137896 tyr__L_c + 0.1 uaagmda_c + 0.144104 utp_c + 0.423162 val__L_c + 0.000341 zn2_c --> 53.95 adp_c + 53.95 h_c 

> Podemos observar que é um meio bem complexo, sugiro começar com um meio mais simples e ir adicionando os componentes conforme a necessidade.

In [3]:
model.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/gustavotamasco/anaconda3/envs/GSMR/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]


{'EX_14glucan_e': 1000.0,
 'EX_25dkglcn_e': 1000.0,
 'EX_2m35mdntha_e': 1000.0,
 'EX_2mbald_e': 1000.0,
 'EX_35dnta_e': 1000.0,
 'EX_3mb_e': 1000.0,
 'EX_4abut_e': 1000.0,
 'EX_4hphac_e': 1000.0,
 'EX_5mdru1p_e': 1000.0,
 'EX_LalaDgluMdap_e': 1000.0,
 'EX_Larab_e': 1000.0,
 'EX_abt_e': 1000.0,
 'EX_ac_e': 1000.0,
 'EX_acald_e': 1000.0,
 'EX_acmum_e': 1000.0,
 'EX_ade_e': 1000.0,
 'EX_ala__L_e': 1000.0,
 'EX_arab__L_e': 1000.0,
 'EX_araban__L_e': 1000.0,
 'EX_arg__L_e': 1000.0,
 'EX_asn__L_e': 1000.0,
 'EX_asp__L_e': 1000.0,
 'EX_biliverd_e': 1000.0,
 'EX_btd_RR_e': 1000.0,
 'EX_btn_e': 1000.0,
 'EX_bz_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_chol_e': 1000.0,
 'EX_cit_e': 1000.0,
 'EX_citr__L_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_co_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_csn_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cys__L_e': 1000.0,
 'EX_cytd_e': 1000.0,
 'EX_dca_e': 1000.0,
 'EX_ddca_e': 1000.0,
 'EX_dextrin_e': 1000.0,
 'EX_etha_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_

In [5]:
from cobra.medium import minimal_medium

> Não sei se o organismo que está estudando consome isso, mas a BOF que está sendo utilizada é para dextrina, que é basicamente amido.


In [6]:
max_growth = model.slim_optimize()
min_media = minimal_medium(model, max_growth)
min_media

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/gustavotamasco/anaconda3/envs/GSMR/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/gustavotamasco/anaconda3/envs/GSMR/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fu

EX_4abut_e             0.001722
EX_4hphac_e            2.374342
EX_LalaDgluMdap_e      1.721836
EX_acmum_e             3.443671
EX_ade_e               3.517658
EX_arg__L_e            5.093052
EX_asn__L_e            4.150571
EX_asp__L_e            4.150571
EX_bz_e                0.001722
EX_ca2_e               0.089622
EX_cl_e                0.089622
EX_cobalt2_e           0.001722
EX_cu2_e               0.012208
EX_dextrin_e         249.611646
EX_fe3_e               0.250062
EX_fol_e               0.011519
EX_gln__L_e            4.535022
EX_glu__L_e            4.377974
EX_gua_e               4.176467
EX_his__L_e            1.631233
EX_icit_e              0.149369
EX_k_e                 3.360903
EX_lys__L_e            5.922443
EX_met__L_e            2.651782
EX_mg2_e               0.149369
EX_mn2_e               0.011898
EX_nac_e               0.039223
EX_no3_e             212.437231
EX_o2_e               48.767158
EX_phe__L_e            3.189959
EX_pi_e               15.257064
EX_pnto_

In [7]:
model.medium = min_media.to_dict()
model.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/gustavotamasco/anaconda3/envs/GSMR/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return most[0]
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
/home/gustavotamasco/anaconda3/envs/GSMR/lib/python3.9/site-packages/cobra/medium/boundary_types.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a fu

{'EX_4abut_e': 0.0017218355696964175,
 'EX_4hphac_e': 2.374342377187797,
 'EX_LalaDgluMdap_e': 1.7218355696955137,
 'EX_acmum_e': 3.4436711393910255,
 'EX_ade_e': 3.5176584138222258,
 'EX_arg__L_e': 5.0930518683168655,
 'EX_asn__L_e': 4.1505707325295305,
 'EX_asp__L_e': 4.150570732529507,
 'EX_bz_e': 0.0017218355696955133,
 'EX_ca2_e': 0.0896215414026515,
 'EX_cl_e': 0.08962154140265148,
 'EX_cobalt2_e': 0.0017218355696955135,
 'EX_cu2_e': 0.01220781418914119,
 'EX_dextrin_e': 249.61164572674082,
 'EX_fe3_e': 0.2500621797858747,
 'EX_fol_e': 0.011519079961262984,
 'EX_gln__L_e': 4.5350221785311335,
 'EX_glu__L_e': 4.377973556219349,
 'EX_gua_e': 4.17646713949774,
 'EX_his__L_e': 1.6312325820181357,
 'EX_icit_e': 0.1493692356710858,
 'EX_k_e': 3.360902503555763,
 'EX_lys__L_e': 5.9224428438803844,
 'EX_met__L_e': 2.651781742532363,
 'EX_mg2_e': 0.1493692356710858,
 'EX_mn2_e': 0.011897883786595996,
 'EX_nac_e': 0.039223414277605964,
 'EX_no3_e': 212.43723115049664,
 'EX_o2_e': 48.767157

In [8]:
model.optimize()

,fluxes,reduced_costs
EX_14glucan_e,0.000000e+00,-8.811278e-02
EX_25dkglcn_e,0.000000e+00,-5.422325e-02
EX_2m35mdntha_e,0.000000e+00,0.000000e+00
EX_2mbald_e,-9.308561e-12,-7.991862e-17
EX_35dnta_e,0.000000e+00,0.000000e+00
...,...,...
r0963,1.151908e-02,0.000000e+00
r2465_1,-5.871459e-03,-0.000000e+00
sink_aacald_c,0.000000e+00,0.000000e+00
Growth,1.721836e+01,0.000000e+00
